In [17]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
import os
import random
import glob
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import kagglehub
from PIL import Image

In [18]:
path = kagglehub.dataset_download("akash2sharma/tiny-imagenet")
print(f"Dataset downloaded to: {path}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("CUDA not available - running on CPU")

Dataset downloaded to: C:\Users\Vaibhava\.cache\kagglehub\datasets\akash2sharma\tiny-imagenet\versions\1
Device: cuda
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU
GPU Memory: 4.0 GB


In [19]:
class TinyIMGNETtst(Dataset):
    def __init__(self,test_path, transform=None):
        self.image_paths = glob.glob(os.path.join(test_path, "*.JPEG"))
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image = Image.open(self.image_paths[index]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [20]:
class AlexNET(nn.Module):
    def __init__(self,input_dim=224):
        super().__init__()
        self.L1=nn.Sequential(nn.Conv2d(in_channels=3,out_channels=96,kernel_size=11,stride=4),nn.ReLU(),nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75,k=2.0),nn.MaxPool2d(kernel_size=3,stride=2))
        self.L2=nn.Sequential(nn.Conv2d(in_channels=96,out_channels=256,kernel_size=5,stride=1),nn.ReLU(),nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75,k=2.0),nn.MaxPool2d(kernel_size=3,stride=2))
        self.L3=nn.Sequential(nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,stride=1,padding=1),nn.ReLU())
        self.L4=nn.Sequential(nn.Conv2d(in_channels=384,out_channels=256,kernel_size=3,stride=1,padding=1),nn.ReLU(),nn.MaxPool2d(kernel_size=3,stride=2),nn.AdaptiveAvgPool2d((6, 6)))
        self.L5=nn.Sequential(nn.Linear(in_features=256*6*6,out_features=4096),nn.ReLU(),nn.Dropout(p=0.5))
        self.L6=nn.Sequential(nn.Linear(in_features=4096,out_features=1000),nn.ReLU(),nn.Dropout(p=0.5))
        self.Lout=nn.Sequential(nn.Linear(in_features=1000,out_features=200))
    def forward(self,X):
        out=self.L2(self.L1(X))
        out=self.L4(self.L3(out))
        out=torch.flatten(out,1)
        out=self.L6(self.L5(out))
        out=self.Lout(out)
        return out

In [21]:
train_path = os.path.join(path, "tiny-imagenet-200/train")
test_path = os.path.join(path, "tiny-imagenet-200/test/images")
train_images = os.listdir(train_path)
test_images = os.listdir(test_path)
transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor()])
train_dataset = ImageFolder(root=train_path, transform=transform)
test_dataset = TinyIMGNETtst(test_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [22]:
l_type=nn.CrossEntropyLoss()
lr=0.0001
model=AlexNET()
optim=torch.optim.Adam(model.parameters(),lr,weight_decay=1e-5)
num_epochs=25
model=model.to(device)

In [24]:
for i in range(num_epochs):
    model.train()
    running_loss=0
    for images,labels in train_loader:
        images,labels=images.to(device),labels.to(device)
        outputs=model(images)
        loss=l_type(outputs,labels)
        optim.zero_grad()
        loss.backward()
        optim.step()
        running_loss+=loss.item()
    print(f"Epoch[{i+1}/{num_epochs}],Loss:{running_loss/len(train_loader):.4f}")    

Epoch[1/25],Loss:4.2252


KeyboardInterrupt: 

In [ ]:
print(len(test_loader))

313


In [ ]:
model.eval()
correct=0
total=0
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs=model(images)
        print(outputs)


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same